**IMPORTANT: BEFORE YOU DO ANYTHING ON THIS COLAB, GO TO FILE AND CLICK "SAVE A COPY IN DRIVE", THEN WORK IN THAT NEW COLAB NOTEBOOK.**

The following link will take you to the required dataset: https://drive.google.com/file/d/16GTzmh4_RHT9yfVUjSowtqtT-wabjlFr/view?usp=sharing.

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import sklearn as sk

## Import Dataset

In [ ]:
df_bnb = pd.read_csv('bootcamptrain2025.csv')
df_bnb.head()

# Data Preparation
Data Integration

First, let's practice data integration with a simple dataset.

In [ ]:
# Create the data for the datasets
a = {'Age': [19, 21], 'Height': [75, 68]}
b = {'age': [22, 30], 'ht': [65, 70]}

In [ ]:
# Produce and show the datasets with pandas
df1 = pd.DataFrame(a)
df1

In [ ]:
# Same thing here
df2 = pd.DataFrame(b)
df2

What if we wanted to combine these two datasets into one? Notice how the column names are not the same in both.

In [ ]:
# let's rename the second dataset to match the first
df2_renamed = df2.rename(columns={'age': 'Age', 'ht': 'Height'})

# use pd.concat to combine two datasets vertically
df3 = pd.concat([df1, df2_renamed])
df3

Let's create a similar situation but with 3 different variables in play.

In [ ]:
# creating two datasets, students and schools
students = {'Name': ['Alice', 'Bob'], 'StudentID': [1, 2]}
schools = {'StudentID': [1, 2], 'School': ['North High', 'East High']}

df_students = pd.DataFrame(students)

df_schools = pd.DataFrame(schools)

In [ ]:
df_students

In [ ]:
df_schools

In [ ]:
# use pd.merge to combine datasets horizontally, aligned by a common variable, in this case 'StudentID'
students_and_schools = pd.merge(df_students, df_schools, on='StudentID')
students_and_schools

Data Profiling

Let's do some data profiling to get basic information about the data's rows, columns, and contents.

In [ ]:
# .shape produces (rows, columns) of a dataset
print(df_bnb.shape)

In [ ]:
# .columns produces a list with all columns of a dataset
print(df_bnb.columns)

Below, we are going to make a basic column summary with data types, missing or unique counts, min, max, mean, and deviation of each column. While it may look complicated or intimidating, most of it is just knowing how to extract data from a specific column.

In [ ]:
# first create the dictionary that will be our dataframe
summary = []

# make a for loop so we collect the same information for every column
for col in df_bnb.columns:
    # here we use various pandas commands to get our information
    col_data = df_bnb[col]
    dtype = col_data.dtype
    non_null_count = col_data.count()
    total = len(col_data)
    missing = total - non_null_count
    missing_pct = (missing / total) * 100
    unique = col_data.nunique()

    # now we add that information to a dictionary called stats
    stats = {
        'Column': col,
        'Data Type': dtype,
        'Non-Null Count': non_null_count,
        '% Missing': round(missing_pct, 2),
        'Unique Values': unique,
    }

    # here we create the min, max, mean, and std dev of our column in stats, only if the data is numerical
    if dtype == int or dtype == float:
        stats['Min'] = col_data.min()
        stats['Max'] = col_data.max()
        stats['Mean'] = col_data.mean()
        stats['Std Dev'] = col_data.std()
    else:
        stats['Min'] = stats['Max'] = stats['Mean'] = stats['Std Dev'] = None

    # finally, we add our stats dictionary to the final summary dictionary
    summary.append(stats)

column_summary_df = pd.DataFrame(summary)

In [ ]:
# this makes the dataframe display with 3 decimal places
pd.set_option('display.float_format', '{:.3f}'.format)
column_summary_df

## Column Renaming

In [ ]:
# Rename columns
df_bnb.rename(columns={'host id': 'host_id', 'NAME':'name', 'host name': 'host_name', 'neighbourhood group': 'neighborhood_group', 'country code': 'country_code', 'room type':'room_type', 'Construction year': 'construction_year', 'service fee':'service_fee', 'minimum nights':'minimum_nights', 'number of reviews':'num_of_reviews','last review':'last_review', 'reviews per month':'reviews_per_month', 'review rate number':'review_rate_number', 'calculated host listings count': 'host_listings_count', 'availability 365':'availability_365'}, inplace=True)

print(df_bnb.columns)

# Data Cleaning

## Nulls

In [ ]:
# Check number of nulls in every column
df_bnb.isnull().sum()

In [ ]:
## Drop license column as almost no rows have value there
df_bnb.drop('license', axis=1, inplace=True)
df_bnb.head()

In [ ]:
# Replace null reviews per month with '0'
df_bnb['reviews_per_month'].fillna(0, inplace=True)
df_bnb.head()

In [ ]:
# Drop House Rules Column
df_bnb.drop('house_rules', axis=1, inplace=True)
df_bnb.head()

In [ ]:
## Replace last_review column with binary has_reviews column
df_bnb['has_reviews'] = np.where(df_bnb['last_review'].isnull(), 0, 1)
df_bnb.drop('last_review', axis=1, inplace=True)
df_bnb.head()

In [ ]:
df_bnb.isnull().sum()

We have a lot of nulls for the country column, but we know what that value should be for all rows.

In [ ]:
# Let's filter the dataset to show rows where country is null
df_bnb[df_bnb['country'].isnull()]

In [ ]:
# We know that the country is US for all apartments, so let's convert the nulls
df_bnb['country'] = df_bnb['country'].fillna("United States")
df_bnb.isnull().sum()

# Exploratory Data Analysis

Now, let's do some basic EDA to get a feel for patterns in our data, starting with a correlation matrix.

In [ ]:
# .corr is a command that makes a correlation matrix for you with every column
corr_matrix = df_bnb.corr(numeric_only=True)
corr_matrix

In [ ]:
# we use matplotlib and seaborn to create a heatmap and make the presentation look nice
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

Let's make some side-by-side boxplots of price categorized by neighborhood. But first, we have to make the 'neighbourhood_group' column is clean.

In [ ]:
# check unique values in column
df_bnb['neighborhood_group'].unique()

In [ ]:
# fix the mistakenly repeated neighborhoods
df_bnb['neighborhood_group'] = df_bnb['neighborhood_group'].replace({
    'brookln': 'Brooklyn',
    'manhatan': 'Manhattan'
})

In [ ]:
# once again, use matplotlib and seaborn to make our boxplots
sns.boxplot(x='neighborhood_group', y='price', data=df_bnb)
plt.title('Price by Neighborhood Group')
plt.ylabel('Price')
plt.xlabel('Neighborhood Group')
plt.show()

Finally, let's make a scatterplot of price and average rating.

In [ ]:
# this time use .scatterplot with seaborn to make the plot
sns.scatterplot(x='average_rating', y='price', data=df_bnb)
plt.title('Price by Reviews Per Month')
plt.ylabel('Price')
plt.xlabel('Average Rating')
plt.show()

# Data Transformation

Now, we're going to perform so simple data transformation. First, we'll look at an example of label encoding and an example of one-hot encoding.

Label Encoding

Let's make room type into a quantitative variable with label encoding.

In [ ]:
# view every different unique entry in room type column
df_bnb['room_type'].unique()

In [ ]:
# make a new column room_type_num and use .replace to replace every unique entry with a corresponding number
df_bnb['room_type_num'] = df_bnb['room_type'].replace({
    'Private room': 1,
    'Entire home/apt': 2,
    'Shared room': 3,
    'Hotel room': 4
})
df_bnb.head()

One-Hot Encoding

Let's do the same thing with the neighborhood group column, except this time using one-hot encoding.

In [ ]:
# .get_dummies produces a dataframe with a column for each unique entry and a 0 or 1 to signify its presence
dummies = pd.get_dummies(df_bnb['neighborhood_group'], prefix='neighborhood_group', dtype=int)

# then use .concat to add the new dataframe to our existing one
df_bnb = pd.concat([df_bnb, dummies], axis=1)
df_bnb.head()

Normalization & Standardization

Now let's go through some examples of normalization and standardization with the "reviews per month" column.

In [ ]:
# all this does is make sure every column is displayed when viewing the data
pd.set_option('display.max_columns', None)
df_bnb

In [ ]:
# make the standardized column using the formula (x - mean)/std
df_bnb['reviews_per_month_standardized'] = (df_bnb['reviews_per_month'] - df_bnb['reviews_per_month'].mean())/df_bnb['reviews_per_month'].std()

# here we print the mean to check if our calculation makes sense
print(df_bnb['reviews_per_month'].mean())
df_bnb

In [ ]:
# make the normalized column using formula (x - min)/(max - min)
df_bnb['reviews_per_month_normalized'] = (df_bnb['reviews_per_month'] - df_bnb['reviews_per_month'].min())/(df_bnb['reviews_per_month'].max() - df_bnb['reviews_per_month'].min())

# printing min and max to check correctness
print(df_bnb['reviews_per_month'].min())
print(df_bnb['reviews_per_month'].max())
df_bnb.head(15)

Feature Extraction

Finally, let's make our own variable using some of our existing ones.

In [ ]:
# create new column "popularity_score" by multiplying number of reviews by reviews per month
df_bnb['popularity_score'] = df_bnb['num_of_reviews'] * df_bnb['reviews_per_month']
df_bnb.head()